In [1]:
import pandas as pd
import sqlite3

import pygwalker as pyg

from datetime import datetime

# TEC CSV database - all contribs CSV files converted to SQLite DB
contribs_db = "20230801194240_TEC contribs.db"


In [2]:
texas_legislators_df = pd.read_csv("InfiniteTransparency - txlege.csv")

print(texas_legislators_df.head())

       Member Name Freshman  District # Chamber Years Served Sessions Served  \
0     Abel Herrero      NaN          34       H  2013 - 2025     83rd - 88th   
1       Alma Allen      NaN         131       H  2005 - 2025     79th - 88th   
2    Ana Hernandez        v         143       H  2005 - 2025     79th - 88th   
3  Ana-Maria Ramos      NaN         102       H  2019 - 2025     86th - 88th   
4      Andrew Murr      NaN          53       H  2015 - 2025     84th - 88th   

  Party        City  County  Twitter Handle  Filer ID  
0     D    Robstown  Nueces  RepAbelHerrero     54318  
1     D     Houston  Harris   RepAllenTX131     19673  
2     D     Houston  Harris        AnaHdzTx     57411  
3     D  Richardson  Dallas     Ramos4Texas     81730  
4     R    Junction  Kimble             NaN     69565  


In [3]:
texas_legislators = texas_legislators_df["Member Name"].unique()

print(texas_legislators)

['Abel Herrero' 'Alma Allen' 'Ana Hernandez' 'Ana-Maria Ramos'
 'Andrew Murr' 'Angela Paxton' 'Angelia Orr' 'Angie Chen Button'
 'Ann Johnson' 'Armando Martinez' 'Armando Walle' 'Barbara Gervin-Hawkins'
 'Benjamin Bumgarner' 'Bob Hall' 'Borris Miles' 'Brad Buckley'
 'Brandon Creighton' 'Brian Birdwell' 'Brian Harrison' 'Briscoe Cain'
 'Brooks Landgraf' 'Bryan Hughes' 'Candy Noble' 'Carl H. Tepper'
 'Carl Sherman, Sr.' 'Carol Alvarado' 'Caroline Harris' 'Carrie Isaac'
 'Cecil Bell, Jr.' 'César Blanco' "Charles 'Doc' Anderson"
 'Charles Cunningham' 'Charles Perry' 'Charles Schwertner' 'Charlie Geren'
 'Chris Turner' 'Christian Manuel' 'Christina Morales' 'Claudia Ordaz'
 'Cody Harris' 'Cody Vasut' 'Cole Hefner' 'Craig Goldman' 'Dade Phelan'
 'David Cook' 'David Spiller' 'Dennis Paul' 'DeWayne Burns' 'Diego Bernal'
 'Donna Campbell' 'Donna Howard' 'Drew Darby' 'Drew Springer'
 'Dustin Burrows' 'Ed Thompson' 'Eddie Morales' 'Ellen Troxclair'
 'Erin Elizabeth Gámez' 'Erin Zwiener' 'Ernest B

In [4]:
# WANT: filerIdent, formTypeCd, filerName

def sqlite_to_df(database, table_name):
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(database)
    
    # Write a SQL query to select all records from the table
    query = f'SELECT * FROM {table_name} WHERE "filerTypeCd" = "COH"'
    
    # Use pandas to run the SQL query and load the result into a DataFrame
    df = pd.read_sql_query(query, conn)
    
    # Close the connection to the SQLite database
    conn.close()
    
    # Return the DataFrame
    return df

df = sqlite_to_df(contribs_db, "contribs")

print(df.head())

  recordType formTypeCd schedFormTypeCd  reportInfoIdent  receivedDt  \
0       RCPT        COH              A1           538277  20120716.0   
1       RCPT        COH              A1           498895  20110715.0   
2       RCPT        COH              A1           559912  20130115.0   
3       RCPT        COH              A1           498895  20110715.0   
4       RCPT     CORCOH              A1           470994  20101215.0   

  infoOnlyFlag  filerIdent filerTypeCd                      filerName  \
0            N       55499         COH  Aliseda, Jose (The Honorable)   
1            N       55499         COH                  Aliseda, Jose   
2            N       55499         COH            Aliseda, Jose (Mr.)   
3            N       55499         COH                  Aliseda, Jose   
4            N       55499         COH            Aliseda, Jose (Mr.)   

   contributionInfoId  ...  contributorStreetRegion  contributorEmployer  \
0           109697898  ...                     None 

In [5]:
print(df.shape[0])


2763824


In [6]:
txlege_tec_filer_ids = texas_legislators_df["Filer ID"].unique()

In [7]:
current_txlege_all_historical_contribs = df[df["filerIdent"].isin(txlege_tec_filer_ids)]

In [8]:
print(current_txlege_all_historical_contribs.shape[0])

605239


In [9]:
master_txlege_contribs_df = pd.merge(current_txlege_all_historical_contribs, texas_legislators_df[["Member Name", "District #", "Chamber", "Party", "City", "County", "Filer ID"]], how="left", left_on="filerIdent", right_on="Filer ID")

In [10]:
print(master_txlege_contribs_df.head())

  recordType formTypeCd schedFormTypeCd  reportInfoIdent  receivedDt  \
0       RCPT        COH              A1           291854  20051202.0   
1       RCPT        COH              A1           361398  20080115.0   
2       RCPT        COH              A1           290394  20051031.0   
3       RCPT        COH              A1           361398  20080115.0   
4       RCPT        COH              A2           290394  20051031.0   

  infoOnlyFlag  filerIdent filerTypeCd                filerName  \
0            Y       57411         COH  Hernandez, Ana E. (Ms.)   
1            N       57411         COH  Hernandez, Ana E. (Ms.)   
2            N       57411         COH  Hernandez, Ana E. (Ms.)   
3            N       57411         COH  Hernandez, Ana E. (Ms.)   
4            N       57411         COH  Hernandez, Ana E. (Ms.)   

   contributionInfoId  ...  contributorSpouseLawFirmName  \
0           109901468  ...                          None   
1           109901469  ...                  

In [11]:
import math

def convert_to_datetime(s):
    try:
        if s is not None and type(s) is float and math.isnan(s) is not True:
            # Convert to number then to integer
            s2 = str(int(s))
            
            # Parse as a date
            d = datetime.strptime(s2, "%Y%m%d")
        
            return d
        else:
            return None
    except Exception as e:
        print(e, s, type(s))

master_txlege_contribs_df["receivedDt"] = master_txlege_contribs_df["receivedDt"].apply(convert_to_datetime)
master_txlege_contribs_df["contributionDt"] = master_txlege_contribs_df["contributionDt"].apply(convert_to_datetime)

print(master_txlege_contribs_df.head())

  recordType formTypeCd schedFormTypeCd  reportInfoIdent receivedDt  \
0       RCPT        COH              A1           291854 2005-12-02   
1       RCPT        COH              A1           361398 2008-01-15   
2       RCPT        COH              A1           290394 2005-10-31   
3       RCPT        COH              A1           361398 2008-01-15   
4       RCPT        COH              A2           290394 2005-10-31   

  infoOnlyFlag  filerIdent filerTypeCd                filerName  \
0            Y       57411         COH  Hernandez, Ana E. (Ms.)   
1            N       57411         COH  Hernandez, Ana E. (Ms.)   
2            N       57411         COH  Hernandez, Ana E. (Ms.)   
3            N       57411         COH  Hernandez, Ana E. (Ms.)   
4            N       57411         COH  Hernandez, Ana E. (Ms.)   

   contributionInfoId  ... contributorSpouseLawFirmName  \
0           109901468  ...                         None   
1           109901469  ...                         N

In [12]:
print(master_txlege_contribs_df.shape[0])

605239


In [13]:
master_txlege_contribs_df.to_excel("Current TxLege All Historic Contributions.xlsx", index=False)

# Assuming `df` is your DataFrame
sample_df = master_txlege_contribs_df.sample(n=5000)  # Generates a random sample of 5000 rows from the DataFrame
sample_df.to_csv('sample_data.csv', index=False)


In [20]:
def generate_html_dropdown_updated_v4(data, columns, output_file):
    # Define the basic structure of the HTML file
    html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>88th TXLege - Dark Money in Pub Ed</title>
        <!-- Include Bootstrap CSS and JS from a CDN -->
        <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/css/bootstrap.min.css">
        <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
        <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.0/js/bootstrap.min.js"></script>
    </head>
    <body>
        <div class="container">
            <h1 class="my-4">88th TXLege - Dark Money in Pub Ed</h1>
            <select id="membersDropdown" class="form-control" onchange="openModal(this.value)">
    """

    # Sort the data by chamber, party, and Member Name
    sorted_data = data.sort_values(by=['Chamber', 'Party', 'Member Name'])

    # Add each member to the HTML as an option in the dropdown menu
    for member in sorted_data['Member Name'].unique():
        member_id = re.sub('\W+','', member)  # Replace all non-alphanumeric characters with ''
        member_info = sorted_data[sorted_data['Member Name'] == member].iloc[0]
        member_detail = f"{member} - {member_info['Chamber']} {member_info['District #']} - {member_info['Party']}"
        html += f"""
                <option value="{member_id}">{member_detail}</option>
        """

    # Close the select, div and body tags
    html += """
            </select>
        </div>
    </body>
    </html>
    """

    # Add a modal for each member
    for member in sorted_data['Member Name'].unique():
        member_id = re.sub('\W+','', member)  # Replace all non-alphanumeric characters with ''
        # Filter the data for the current member
        member_data = sorted_data[sorted_data['Member Name'] == member]

        # Calculate the sum of the contributions
        total_contributions = "{:,}".format(member_data['contributionAmount'].sum())

        # Sort the contributions in descending order of the first column
        member_data = member_data.sort_values(by=columns[0], ascending=False)

        # Create a HTML table with the member's contributions
        table = member_data[columns].to_html(index=False, classes='table table-striped')

        member_info = member_data.iloc[0]
        member_detail = f"{member} - {member_info['Chamber']} {member_info['District #']} - {member_info['Party']}"

        # Add the modal to the HTML
        html = html.replace("</body>", f"""
        <!-- {member} Modal -->
        <div class="modal fade" id="{member_id}Modal" tabindex="-1" role="dialog" aria-labelledby="{member_id}ModalLabel" aria-hidden="true">
            <div class="modal-dialog modal-xl" role="document">
                <div class="modal-content">
                    <div class="modal-header">
                        <h5 class="modal-title" id="{member_id}ModalLabel">{member_detail}'s Contributions - Total: ${total_contributions}</h5>
                        <button type="button" class="close" data-dismiss="modal" aria-label="Close">
                            <span aria-hidden="true">&times;</span>
                        </button>
                    </div>
                    <div class="modal-body">
                        {table}
                    </div>
                    <div class="modal-footer">
                        <button type="button" class="btn btn-secondary" data-dismiss="modal">Close</button>
                    </div>
                </div>
            </div>
        </div>
        </body>
        """)

    # Add a script to open the modal when a member is selected
    html = html.replace("</body>", f"""
    <script>
        function openModal(member) {{
            $('#'+member+'Modal').modal('show');
        }}
    </script>
    </body>
    """)

    # Save the HTML to a file
    with open(output_file, 'w') as f:
        f.write(html)

# Define the columns to include in the tables
columns = ['receivedDt', 'contributionAmount', 'contributorNameOrganization', 'contributionInfoId', 'reportInfoIdent']

master_txlege_contribs_df = master_txlege_contribs_df[master_txlege_contribs_df['contributorNameOrganization'].notnull()]

# Use the function to generate the HTML file
# generate_html_dropdown_updated_v2(master_txlege_contribs_df, columns, "index.html")


In [21]:
# check to make sure donations are not double-counted, and 
# only load corrected reports if one is available

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def find_similar(df, column, input_str, threshold):
    """
    Find all values in a DataFrame column that are similar to an input string.

    Parameters:
    - df: DataFrame to search
    - column: column in df to search
    - input_str: string to search for
    - threshold: minimum similarity score to consider a match

    Returns:
    - DataFrame with matches and similarity scores
    """
    # Initialize list to hold tuples of matches and scores
    matches = []

    # Iterate over the column
    for value in df[column].unique():
        # Calculate similarity score
        score = fuzz.ratio(input_str, str(value))

        # If the score is above the threshold, append to matches
        if score >= threshold:
            matches.append((value, score))

    # Create a DataFrame from the matches and sort by score
    matches_df = pd.DataFrame(matches, columns=[column, 'Score'])
    matches_df = matches_df.sort_values('Score', ascending=False)

    return matches_df


In [22]:
# similar_contributors = find_similar(master_txlege_contribs_df, 'contributorNameOrganization', 'educational freedom', 55)

# print(list(similar_contributors["contributorNameOrganization"]))

dark_money_donors = {
    "Charter Schools Now PAC": ['Charter Schools Now PAC', 'Charter School Now PAC', 'TX Charter Schools Now PAC', 'Tx Charter Schools Now PAC', 'Charter Schools NOW PAC', 'Charter Schools Now Pac', 'Charter Schools Now', ' Charter Schools Now', 'Texas Charter Schools Now PAC', 'Texas Charter Schoools Now PAC', 'Texas Charter School Now PAC', 'Charter Schools Now PAC - Texas', 'Texas Charter Schools Now Pac', 'Texas Charter Schools NOW PAC', 'Texas Charter Schools Now', 'Texas Charter Schools PAC', 'Texas Charter School Now Pac', 'The Texas Charter Schools NOW PAC', 'Charter Schools Now Political Action Committee', 'Texas Charter Schools Association', 'Texas Charter Schools Now Political Action Committee', ' Texas Charter Schools Association', 'Texas Charter Schools NOW Political Action Committee', 'TCSN PAC'],
    "Legacy 44": ['Legacy 44', 'Legacy #44', 'Legacy 44 PAC', 'Legacy 44 (PAC)'],
    "Leadership for Educational Equity": ['Leadership for Educational Equity', 'Leadership for Educational Equity - Texas', 'Allies for Educational Equity',],
    "Democrats for Education Reform": ['Democrats for Education Reform', 'Texans for Education Reform', 'Texans For Education Reform', 'Texas for Education Reform PAC', 'Texans for Education Reform PAC', 'Texas For Education Reform PAC', 'Texasns for Education Reform PAC', 'Texans for Educational Reform PAC', 'Texans For Education Reform PAC', 'Democrats for Educational Equity PAC', 'Texas Education Reform PAC',],
    "Texans for Educational Freedom/FFOT": ['Texas Federation for Children PAC', ' Texas Federation for Children PAC', 'Texas Federation for Children PAC, Inc.'],
    "Family Empowerment Coalition": ['Family Empowerment Coalition PAC'],
}

In [23]:
master_contributor_filter = []

import re

for org, names in dark_money_donors.items():
    for name in names:
        master_contributor_filter.append(name)

master_txlege_contribs_df = master_txlege_contribs_df[master_txlege_contribs_df['contributorNameOrganization'].isin(master_contributor_filter)]

# Use the function to generate the HTML file
generate_html_dropdown_updated_v4(master_txlege_contribs_df, columns, "index.html")